In [1]:
import cv2
import time
import numpy as np
import insightface
from insightface.app import FaceAnalysis
from mtcnn import MTCNN
import os

In [2]:
import os
os.chdir("/Users/komangandikawirasantosa/Face-Rec/DeepFace-IPYNB")

In [3]:
os.getcwd()

'/Users/komangandikawirasantosa/Face-Rec/DeepFace-IPYNB'

In [4]:
app = FaceAnalysis(allowed_modules=['detection', 'recognition'])
app.prepare(ctx_id=0, det_size=(640, 640))

/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CoreMLExecutionProvider, AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 

In [5]:
# Define zones
frame_height, frame_width = 480, 640
zones = {
    "Top-Left": (0, 0, frame_width // 2, frame_height // 2),
    "Top-Right": (frame_width // 2, 0, frame_width, frame_height // 2),
    "Bottom-Left": (0, frame_height // 2, frame_width // 2, frame_height),
    "Bottom-Right": (frame_width // 2, frame_height // 2, frame_width, frame_height),
}


In [10]:
database = {}
path_to_images = "Database/" 

In [7]:
for filename in os.listdir(path_to_images):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        filepath = os.path.join(path_to_images, filename)
        image = cv2.imread(filepath)
        faces = app.get(image)
        
        if len(faces) > 0:
            # Use the first detected face for embedding
            embedding = faces[0].embedding
            database[filename.split(".")[0]] = embedding

In [8]:
# Function to get the zone based on face coordinates
def get_zone(x, y, w, h):
    centroid_x, centroid_y = x + w // 2, y + h // 2
    for zone_name, (x1, y1, x2, y2) in zones.items():
        if x1 <= centroid_x <= x2 and y1 <= centroid_y <= y2:
            return zone_name
    return "Unknown"

In [11]:
# Initialize video capture
cap = cv2.VideoCapture(1)
capture_interval = 3  # seconds
last_capture_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    current_time = time.time()
    if current_time - last_capture_time >= capture_interval:
        last_capture_time = current_time  # Update capture time

        # Detect faces in the frame using InsightFace
        faces = app.get(frame)
        
        for face in faces:
            x, y, w, h = face.bbox.astype(int)
            embedding = face.embedding
            
            # Match embedding with the database
            name = "Unknown"
            min_distance = float("inf")
            
            for db_name, db_encoding in database.items():
                distance = np.linalg.norm(db_encoding - embedding)
                if distance < min_distance and distance < 0.7:  # Set a threshold for recognition
                    min_distance = distance
                    name = db_name
            
            # Identify zone
            zone = get_zone(x, y, w, h)
            
            # Draw box and label on the frame
            cv2.rectangle(frame, (x, y), (w, h), (0, 255, 0), 2)
            cv2.putText(frame, f"{name} - {zone}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

        # Display the frame with annotations
        cv2.imshow("Zone Identification", frame)

    # Check for exit key
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
import cv2
import numpy as np
import os
from insightface import app
from scipy.spatial.distance import cosine

# Load the InsightFace model
face_recognition = app.FaceAnalysis()
face_recognition.prepare(ctx_id=0, det_size=(640, 640))

# Load the face database
database_path = 'Database'  # Replace with the path to your database
face_embeddings = {}
for person_name in os.listdir(database_path):
    person_folder = os.path.join(database_path, person_name)
    if os.path.isdir(person_folder):
        face_embeddings[person_name] = []
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                faces = face_recognition.get(img)
                for face in faces:
                    # Store the normalized embedding of the first face detected
                    face_embeddings[person_name].append(face.normed_embedding)


/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CoreMLExecutionProvider, AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.in

/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [5]:
cap = cv2.VideoCapture(1)

# Set camera properties for better image quality
cap.set(cv2.CAP_PROP_BRIGHTNESS, 0.5)  # Adjust brightness (range: 0.0 - 1.0)
cap.set(cv2.CAP_PROP_CONTRAST, 0.5)    # Adjust contrast (range: 0.0 - 1.0)
cap.set(cv2.CAP_PROP_EXPOSURE, 0.1)     # Adjust exposure (may vary by camera)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Perform face detection and recognition
    faces = face_recognition.get(frame)

    # Display results
    for face in faces:
        # Get bounding box and landmarks
        bbox = face.bbox.astype(int)
        identity = face.normed_embedding

        # Compare with the database
        recognized_person = None
        min_distance = float('inf')
        for person_name, embeddings in face_embeddings.items():
            for embedding in embeddings:
                distance = cosine(identity, embedding)
                if distance < min_distance:
                    min_distance = distance
                    recognized_person = person_name

        # Draw bounding box and label on the image
        if recognized_person is not None and min_distance < 0.5:  # Adjust threshold as necessary
            label = recognized_person
        else:
            label = "Unknown"

        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
        cv2.putText(frame, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the resulting frame
    cv2.imshow('Face Recognition', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()